In [2]:
from scapy.all import *
import csv
import os
import scapy
from scapy.all import rdpcap
from scapy.all import PcapReader, IP, IPv6, TCP, UDP, Raw, Ether
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import defaultdict
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, InputLayer
import os 
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model
import pandas as pd

In [3]:
dataset_path = 'input_dataset'

categorized_filenames = {
    'video_stream': [],
    'messaging': [],
    'file_transfer': [],
    'regular_browsing': []
}

In [4]:
def is_tls_handshake(packet):
    if packet.haslayer(TCP) and packet[TCP].dport == 443:
        try:
            raw = packet[TCP].load
            if raw and raw.startswith(b'\x16\x03'):
                # TLS Handshake type
                return 'Yes'
        except Exception as e:
            pass
    return 'No'

In [5]:
from tensorflow.keras.models import load_model
model = load_model("neural_net_payload.h5")

In [6]:
def payload_to_decimal(packet):
    decimal_payload = []
    if Raw in packet:
        payload = packet[Raw].load
        decimal_payload = [int(b) for b in payload]
    return decimal_payload

In [7]:
# def extract_data(application_type, file_name, data_size):
#     packets = rdpcap(f"New_Data/{file_name}")
#     for packet in packets[:data_size]:
#         try:
#             if Raw in packet:
#                 decimal_vectors =  payload_to_decimal(packet)
#             else:
#                 decimal_vectors = []
#             input_data.append(decimal_vectors)
#             application_types.append(application_type)
#         except Exception as e:
#             print(e)

In [8]:
def retrain(decimal_payload):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[2].output)
    max_length = 20
    input_data = decimal_payload[:20] if len(decimal_payload) > 20 else decimal_payload
    single_input = pad_sequences([decimal_payload], maxlen=max_length, padding='post')
    output_from_last_hidden_layer = intermediate_layer_model.predict(single_input)
    return output_from_last_hidden_layer

In [9]:
def run_for_all(pcap_file):
    packets=rdpcap(pcap_file)
    with open(output_csv_file, 'a', newline='') as csvfile:
        fieldnames = ['Timestamp', 'Start Timestamp', 'TLS Handshake', 'Source IP', 'Destination IP', 'Port', 'Retrain','Label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        if 'netflix' in pcap_file or 'youtube' in pcap_file:
            label = 'video_stream'
        elif 'skype' in pcap_file:
            label = 'messaging'
        elif 'sftp' in pcap_file:
            label = 'file_transfer'
        elif 'regular_browsing' in pcap_file:
            label = 'regular_browsing'
        else:
            print(pcap_file)
        
        #print(label,pcap_file, len(packets))
        first_packet = True
        count = 0
        # Iterate through packets and write relevant info to the CSV
        for packet in packets[:2000]:
            count = count + 1
            #print(count, pcap_file)
            if IP in packet:
                timestamp = packet.time
                if first_packet:
                    start_timestamp = packet.time
                    first_packet = False
                tls_handshake = is_tls_handshake(packet)
                src_ip = packet[IP].src
                dest_ip = packet[IP].dst
                port = packet[IP].sport if TCP in packet or UDP in packet else ''
                payload=payload_to_decimal(packet)
                output_from_last_hidden_layer=retrain(payload)
                
                writer.writerow({
                    'Timestamp': timestamp,
                    'Start Timestamp' : start_timestamp,
                    'TLS Handshake': tls_handshake,
                    'Source IP': src_ip,
                    'Destination IP': dest_ip,
                    'Port': port, 
                    'Retrain':output_from_last_hidden_layer,
                    'Label': label,
                })

In [11]:
dataset_path = 'input_dataset'
output_csv_file = 'final.csv'
try:
    #os.remove(output_csv_file)
    print(f"{output_csv_file} has been successfully deleted.")
except FileNotFoundError:
    print(f"{output_csv_file} does not exist.")

for filename in os.listdir(dataset_path):
    if 'pcap' in filename:
        pcap_file = os.path.join(dataset_path, filename)
        # print(pcap_file)
        run_for_all(pcap_file)

final.csv has been successfully deleted.


In [ ]:
'''

divide into batches based on time.

features : 
1. number of packets
2. average packet length
4. number of unique dests
5. TCP protocol count
6. UDP protocol count
7. sshv2 protocol count
8. Inter-arrival time variance
9. Packet size variance
10. Entropy

'''